# Topic Modelling in Python with NLTK and Gensim

https://www.computerhope.com/issues/ch001636.htm

In [1]:
import sys

In [2]:
if 'scosy' not in ''.join(sys.path):
    sys.path.extend(['/home/guerramarj/github/scosy', '/home/guerramarj/github/scosy/utils'])
sys.path

['',
 '/home/guerramarj/python/Python-3.6.3',
 '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python36.zip',
 '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6',
 '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/lib-dynload',
 '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages',
 '/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/IPython/extensions',
 '/home/guerramarj/.ipython',
 '/home/guerramarj/github/scosy',
 '/home/guerramarj/github/scosy/utils']

In [6]:
from utils.datapath import data_path
from pathlib import Path
import pandas as pd
import spacy
import nltk
import random
from gensim import corpora
import pickle
import pyLDAvis.gensim
from spacy.lang.en import English
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
import gensim

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/guerramarj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/guerramarj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
def tokenize(text):
    
    parser = English()

    lda_tokens = list()
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
            
    return lda_tokens

In [8]:
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [9]:
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [10]:
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [11]:
path = data_path(project_name='scosy')
path

'/home/guerramarj/github/scosy/dataset'

In [14]:
data = Path(path, filename).open()

In [21]:
text_data = list()
filename = 'results.txt'

with Path(path, filename).open() as result_file:
    for row in result_file:
        print(row)
#     tokens = prepare_text_for_lda(line)
#     if random.random() > .99:
#         print(tokens)
#     text_data.append(tokens)

In [20]:
text_data[:5]

[['innovation',
  'database',
  'management',
  'computer',
  'science',
  'engineering'],
 ['performance', 'prime', 'field', 'multiplication'],
 ['enchant',
  'scissors',
  'scissor',
  'interface',
  'support',
  'cutting',
  'interactive',
  'fabrication'],
 ['detection',
  'channel',
  'degradation',
  'attack',
  'intermediary',
  'linear',
  'network'],
 ['pinning', 'complex', 'network', 'betweenness', 'centrality', 'strategy']]

In [21]:
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]

In [28]:
num_topics = 5
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = num_topics, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.from_iter(generator)) or th

(0, '0.024*"network" + 0.020*"base" + 0.011*"large" + 0.010*"system"')
(1, '0.040*"network" + 0.023*"wireless" + 0.019*"search" + 0.015*"sensor"')
(2, '0.016*"using" + 0.015*"application" + 0.013*"design" + 0.013*"system"')
(3, '0.025*"network" + 0.024*"base" + 0.015*"system" + 0.014*"design"')
(4, '0.027*"query" + 0.020*"using" + 0.020*"base" + 0.014*"video"')


example

In [29]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(125, 1), (234, 1), (442, 1), (587, 1), (1859, 1)]
[(0, 0.033656023), (1, 0.033775568), (2, 0.86461765), (3, 0.034284204), (4, 0.033666555)]


Saliency: a measure of how much the term tells you about the topic.

Relevance: a weighted average of the probability of the word given the topic and the word given the topic normalized by the probability of the topic.

The size of the bubble measures the importance of the topics, relative to the data.

First, we got the most salient terms, means terms mostly tell us about what’s going on relative to the topics. We can also look at individual topic.

In [31]:
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


Visualizing 3 topics:

In [32]:
lda_display3 = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/home/guerramarj/packages/anaconda3/envs/nlp/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
